In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import pickle



data = '../Data/cleaned_data_eda.csv'

df = pd.read_csv(data)

# Checking for null values
df.isnull().sum()



# Checking to see the percentage of null values out of the entire dataset
100 * df.isnull().sum() / len(df)


# The missing amount for year_made is 43%, and the missing amount for made_in is 66%. Since these are such large portions of the dataset and the only way to find that data would be to search for it, and since I don't have accurate reads on the year_made feature - since they're listed as decades I'm going to remove both columns from the dataset
df.drop(['Unnamed: 0', 'year_made', 'made_in'], axis=1, inplace=True)


# The url column also isn't relevant to any model, so I will remove it.
df.drop('urls', axis=1, inplace=True)


# Looking at the synth_types that are null
print(len(df[df['synth_types'].isnull()]))
df[df['synth_types'].isnull()]

re.findall('Nord', df['Description'][133])



# Building a function to try to pull synth types from the description
def synth_type_identifier(row):
    for synth_type in df.synth_types.unique():
        if re.search(str(synth_type).lower(), str(row).lower()):
            return synth_type
        else:
            pass

df['synth_types'].fillna(df['Description'].apply(synth_type_identifier), inplace=True)

df.synth_types.isnull().sum()

# I will remove all remaining null values for synth type
df.dropna(subset=['synth_types'], axis=0, inplace=True)

df.isnull().sum()

df.info()


## ## 2. Feature Engineering

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = df.drop('Price', axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

84
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3514 entries, 0 to 3591
Data columns (total 7 columns):
Brand          3514 non-null object
Condition      3514 non-null object
Description    3514 non-null object
Model          3514 non-null object
synth_types    3514 non-null object
Price          3514 non-null float64
n_keys         3514 non-null object
dtypes: float64(1), object(6)
memory usage: 219.6+ KB


In [17]:
from catboost import CatBoostClassifier

model = CatBoostClassifier()

categorical_features_indices = np.where(X.dtypes != np.float)[0]

model.fit(X_train, y_train, cat_features=categorical_features_indices)
model.score(X_test, y_test)

#pickle.dump(lasso, open('model.pkl', 'wb'))

#model = pickle.load(open('model.pkl', 'rb'))

Learning rate set to 0.082842
0:	learn: 6.1410647	total: 4m 23s	remaining: 3d 1h 12m 31s
1:	learn: 6.0029476	total: 8m 26s	remaining: 2d 22h 14m 38s
2:	learn: 5.9074661	total: 13m 9s	remaining: 3d 52m 41s
3:	learn: 5.8185107	total: 17m 20s	remaining: 2d 23h 58m 45s
4:	learn: 5.7161455	total: 21m 24s	remaining: 2d 23h 1m 49s
5:	learn: 5.6469285	total: 25m 53s	remaining: 2d 23h 30m 42s
6:	learn: 5.5783506	total: 29m 52s	remaining: 2d 22h 38m 49s
7:	learn: 5.5153316	total: 1h 3m 27s	remaining: 5d 11h 8m 23s


KeyboardInterrupt: 